# **Automatiser les rapports de ventes avec SQL**
**Problématique métier**
 Une entreprise souhaite produire un rapport détaillant les ventes
 totales par produit pour le dernier trimestre.
# **Objectif**
Utiliser SQL pour extraire, filtrer et agréger les données nécessaires.

# **IMPORTATION DE LA CLASS DB**

In [21]:
import sqlite3


In [25]:
class Database:
    def __init__(self, db_path):
        """Initialise la connexion à la base de données SQLite."""
        self.db_path = db_path
        self.conn = None
        self.cursor = None

    def connect(self):
        """Établit la connexion à la base de données."""
        self.conn = sqlite3.connect(self.db_path)
        self.cursor = self.conn.cursor()

    def execute_query(self, query, params=None):
        """Exécute une requête (SELECT, INSERT, UPDATE, etc.)."""
        if self.conn is None:
            self.connect()
        try:
            if params:
                self.cursor.execute(query, params)
            else:
                self.cursor.execute(query)
            self.conn.commit()
            return self.cursor.fetchall()
        except sqlite3.Error as e:
            print(f"Erreur SQLite : {e}")
            return None

    def close(self):
        """Ferme la connexion proprement."""
        if self.cursor:
            self.cursor.close()
        if self.conn:
            self.conn.close()
            self.conn = None
            self.cursor = None

In [29]:
db = Database("Chinook_Sqlite.sqlite")
db.connect()


In [30]:
resultat = db.execute_query("SELECT * FROM Artist")

# **Requêtes de base**
**1.**
    **Clients non américains : Requête affichant les**
    **Clients (leurs noms complets, ID client et pays) qui ne sont pas aux**
    **États-Unis**

In [39]:
# Query to select all customers from the USA
db_noUsa = db.execute_query("SELECT CustomerId,FirstName, LastName, Country FROM customer WHERE country NOT IN('USA')")
print(f"Nombre de client qui sont pas des Etat-Unis: {len(db_noUsa)}")


Nombre de client qui sont pas des Etat-Unis: 46
